In [1]:
!pip install ../input/pythongdcm/python_gdcm-3.0.9.0-cp37-cp37m-manylinux2014_x86_64.whl
!pip install ../input/efficientnet-library/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/efficientnet-library/efficientnet-1.1.1-py3-none-any.whl
!pip install ../input/imutils-library/imutils-0.5.4

Processing /kaggle/input/pythongdcm/python_gdcm-3.0.9.0-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/efficientnet-library/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/efficientnet-library/efficientnet-1.1.1-py3-none-any.whl
Processing /kaggle/input/imutils-library/imutils-0.5.4
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=bb03f351d31fecb99e13500d36d5bcc76591daf265c5b73ffb9902c06eeb5315
  Stored in directory: /root/.cache/pip/wheels/53/24/33/4568efa1711ab50ef6650dd96427fc9655987e74cf156e8d79
Successfully built imutils


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import (Input, Dense, Dropout,LSTM,TimeDistributed, MaxPooling2D,Conv2D, 
                                    GlobalAveragePooling2D, Flatten)
from tensorflow.keras.models import Model, load_model
#from tensorflow.keras.applications.resnet50 import ResNet50
import glob
import os
from tqdm import tqdm
import imageio
from sklearn.model_selection import train_test_split
#from sklearn.metrics import roc_auc_score, confusion_matrix
from efficientnet.tfkeras import EfficientNetB2, EfficientNetB1
#from sklearn.preprocessing import OneHotEncoder
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage.transform import resize
import time
from PIL import Image
import warnings 
warnings.filterwarnings('ignore')
import shutil
import cv2
import imutils

In [3]:
IMG_SIZE = 256
N_SEQUENCES = 20

In [4]:
no_image = np.zeros((IMG_SIZE, IMG_SIZE))
imageio.imwrite('./No_image.png', no_image)

In [5]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    #print(data.mean())
    if data.mean() == 0:
        return 'No Img'
    else:
        return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [6]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'
os.makedirs(save_dir, exist_ok=True)

test_cases = os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [7]:
scans = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
for i in tqdm(test_cases):
    os.makedirs(save_dir + i)
    for s in scans:
        os.makedirs(save_dir + i + '/' + s)

100%|██████████| 87/87 [00:00<00:00, 6079.31it/s]


In [8]:
def select_images(images, n_images_to_consider=20):
    '''Selects the given number of images to consider'''
    no_image_path = './No_image.png'
    images_path_100 = []
    n_images = len(images)
    #new_paths_100 = []
    
    if n_images < n_images_to_consider:
        while n_images < n_images_to_consider:
            images.append(no_image_path)
            n_images += 1
        new_paths_100 = images.copy()
    elif n_images == n_images_to_consider:
        new_paths_100 = images.copy()
    else:
        m = n_images - n_images_to_consider
        if m % 2 == 0:
            left = right = int(m/2)
        else:
            left = int(m//2)
            right = int(left + 1)
        
        right = n_images - right
        new_paths_100 = images[left:right]
    
    return new_paths_100

In [9]:
def save_images_case_wise(case, images):
    '''Saves only valid images'''
    
    path = '/kaggle/tmp/test/' + case + '/T1w/'
    new_filenames = []
    for file in images:
        #print(file)
        xray = read_xray(file)

        if xray != 'No Img':
            im = resize(xray, size=IMG_SIZE) 
            
            img_id = file.split('/')[-1].split('.')[-2] + '.png'

            filename = path + img_id
            im.save(filename)
            new_filenames.append(filename)
    
  #  print(len(new_filenames))
    selected_images = select_images(new_filenames, N_SEQUENCES)
  #  print(len(selected_images))
    return selected_images

In [10]:
t1w_cases = dict() # images

for CASE in tqdm(test_cases):

    temp = sorted(glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/'+CASE+'/T1w/*'), 
              key = lambda l: int(l.split('-')[-1].split('.')[-2]))
    
    #c_t2w_images = select_images(temp, N_SEQUENCES)
    
    new_filenames = save_images_case_wise(CASE, temp)
    
    t1w_cases[CASE] = new_filenames

100%|██████████| 87/87 [02:13<00:00,  1.54s/it]


In [11]:
len(t1w_cases['00013']), t1w_cases['00013'][:3]

(20,
 ['/kaggle/tmp/test/00013/T1w/Image-4.png',
  '/kaggle/tmp/test/00013/T1w/Image-5.png',
  '/kaggle/tmp/test/00013/T1w/Image-6.png'])

In [12]:
def apply_clahe(img, cliplimit=10, grid_size=8):
    #img = cv2.imread(img)
    if img.mean() > 0.1:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hsv_planes = cv2.split(hsv)
        clahe = cv2.createCLAHE(clipLimit=cliplimit, tileGridSize=(grid_size,grid_size))
        hsv_planes[2] = clahe.apply(hsv_planes[2])
        hsv = cv2.merge(hsv_planes)
        hsv = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        return hsv, img.shape
    else:
        return "Don't Use", img.shape

def img_crop(img_path):
    
   # IMG_SIZE = (512, 512)
    img, shape = apply_clahe(img_path)  # original
   # img = cv2.resize(img, dsize=IMG_SIZE, interpolation=cv2.INTER_CUBIC)
    #img_clahe = apply_clahe(img)
    if img != "Don't Use":

        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        if cnts == []:
            return 'Max is Null', 'asd'
        else:
            c = max(cnts, key=cv2.contourArea)

            # find the extreme points
            extLeft = tuple(c[c[:, :, 0].argmin()][0])
            extRight = tuple(c[c[:, :, 0].argmax()][0])
            extTop = tuple(c[c[:, :, 1].argmin()][0])
            extBot = tuple(c[c[:, :, 1].argmax()][0])

            # add contour on the image
            img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)  # contour

            # add extreme points
            img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
            img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
            img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
            img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)  # image with extreme points

            # crop
            ADD_PIXELS = 0
            new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy() #cropped

            return new_img, shape
    else:
        return 'No Image', 'No Shape'

In [13]:
def generator_inference(x, batch_size=8, n_sequences = 25):
    
    AUTO = tf.data.experimental.AUTOTUNE
    
    def load_image(x):
    
        file = tf.io.read_file(x)
        img = tf.image.decode_png(file, channels = 3)
        #img = tf.image.convert_image_dtype(img, tf.float16)
        #img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))   

        return img

    def load_image_stream(x):

        image_sequence = np.zeros((n_sequences, IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)
        img_path_lst = t1w_cases[x.decode('utf-8')]
        #img_path_lst = t2w_cases[x]
        #print(img_path_lst)
        c = 0
        for p in img_path_lst:
            img = cv2.imread(p)
            
            clahe_and_cropped, shape = img_crop(img)
            if clahe_and_cropped != 'No Image' and clahe_and_cropped != 'Max is Null':
                if shape[0] > 224:
                    img_resized = cv2.resize(clahe_and_cropped, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
                else:
                    img_resized = cv2.resize(clahe_and_cropped, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
              #  img = apply_clahe(img)/255.0
                #image_sequence[c] = np.resize(img, (IMG_SIZE, IMG_SIZE, 3))
                image_sequence[c] = tf.image.convert_image_dtype(img_resized, tf.float32)
                c += 1
        return image_sequence.astype(np.float32)
    
    dset = tf.data.Dataset.from_tensor_slices((x))
    dset = dset.map(lambda x1: tf.numpy_function(load_image_stream, [x1], [tf.float32]), num_parallel_calls = AUTO)
    #dset = dset.batch(batch_size)
    dset = dset.prefetch(buffer_size = AUTO)
    
    return dset

In [14]:
model0 = load_model('../input/t1w-ensemble-on-256-20-images/T1w_with_20_256_Res_B1_Preprocessed_0_epoch_10.h5')
model1 = load_model('../input/t1w-ensemble-on-256-20-images/T1w_with_20_256_Res_B1_Preprocessed_1_epoch_10.h5')
model2 = load_model('../input/t1w-ensemble-on-256-20-images/T1w_with_20_256_Res_B1_Preprocessed_2_epoch_10.h5')
model3 = load_model('../input/t1w-ensemble-on-256-20-images/T1w_with_20_256_Res_B1_Preprocessed_3_epoch_10.h5')
model4 = load_model('../input/t1w-ensemble-on-256-20-images/T1w_with_20_256_Res_B1_Preprocessed_4_epoch_10.h5')

models = [model0, model1, model2, model3, model4]

In [15]:
test_gen = generator_inference(test_cases, batch_size=4, n_sequences=N_SEQUENCES)

In [16]:
combined_preds = sum([m.predict(test_gen, verbose=1)[:,1] for m in models])/len(models)

87/87 [==============================] - 12s 110ms/step


In [17]:
sample_sub = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sample_sub.head()

,BraTS21ID,MGMT_value
0,1,0.5
1,13,0.5
2,15,0.5
3,27,0.5
4,37,0.5


In [18]:
def modify_cases(x):
    return int(x)

In [19]:
submission = pd.DataFrame()
submission['BraTS21ID'] = test_cases
submission['MGMT_value'] = combined_preds
submission['BraTS21ID'] = submission['BraTS21ID'].apply(lambda x: modify_cases(x))
submission

,BraTS21ID,MGMT_value
0,114,0.455516
1,13,0.453575
2,821,0.442277
3,644,0.458444
4,699,0.415174
...,...,...
82,474,0.441804
83,174,0.419048
84,119,0.450226
85,80,0.458487


In [20]:
os.remove('./No_image.png')

In [21]:
submission.to_csv('submission.csv', index=False)